In [ ]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.6.0


In [ ]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.64.157.154:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.64.157.154:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of accelerators:  8


In [ ]:
BATCH_SIZE = 16*strategy.num_replicas_in_sync
LEARNING_RATE = 0.01
LEARNING_RATE_EXP_DECAY = 0.6 if strategy.num_replicas_in_sync == 1 else 0.7

In [ ]:
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

Running on TPU  ['10.64.157.154:8470']


In [ ]:
strategy.num_replicas_in_sync

8

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import cv2
from imutils import paths
import os
import time
import random
from keras.preprocessing.image import img_to_array

data=[]
labels=[]
i=1

In [ ]:
imagePaths = sorted(list(paths.list_images("/content/drive/MyDrive/Datasets/dataset1")))
print(imagePaths)
for imagePath in imagePaths:
  if i%50==0:
    print(i)
  i+=1
  image = cv2.imread(imagePath)
  image = cv2.resize(image,(50, 50))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  image = img_to_array(image)
  label = imagePath.split(os.path.sep)[-2]
  if label=="with_mask":
    label=0
    d.append((image,label))
  elif label=="without_mask":
    label=1
    d.append((image,label))

To use the same shuffled data for all cases, we are loading the shuffled data which was saved earlier. If not to use the same shuffled data, skip the next code snippet.

In [ ]:
import pickle
with open("/content/drive/MyDrive/Datasets/pnemoniaBinary.txt", "rb") as fp:
  d = pickle.load(fp)

In [ ]:
for i in d:
  data.append(i[0])
  labels.append(i[1])

In [ ]:
import numpy as np
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
train_X = data
train_y = labels
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(data)*VAL_PCT)
print(val_size)

train_X = data[:2*-val_size]
train_y = labels[:2*-val_size]

test_X = data[2*-val_size:-val_size]
test_y = labels[2*-val_size:-val_size]

test_X1 = data[-val_size:]
test_y1 = labels[-val_size:]


158


In [ ]:
EPOCHS = 29
BATCH_SIZE = 128*8
steps_per_epoch = len(train_X)//BATCH_SIZE
print("Steps per epoch: ", steps_per_epoch)  

Steps per epoch:  1


In [ ]:
# This model trains to 99.4% accuracy in 10 epochs (with a batch size of 64)  

def make_model():
    pass
     # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
    return model
    
with strategy.scope():
  model = tf.keras.Sequential(
      [
        
        tf.keras.layers.Conv2D(32, (5, 5), input_shape = (50, 50, 1), activation = 'relu'), # no bias necessary before batch norm
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
        
        tf.keras.layers.Conv2D(64, (5, 5), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
        
        tf.keras.layers.Conv2D(128, (5, 5), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = 512, activation = 'relu'),
        
        tf.keras.layers.Dense(1, activation='sigmoid')
      ])

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# print model layers
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 46, 46, 32)        832       
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 19, 19, 64)        51264     
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 5, 5, 128)         204928    
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 512)             

In [ ]:
from tensorflow.keras.callbacks import *
mc=ModelCheckpoint('plant_gpu.h5', monitor='val_accuracy', mode='max', save_best_only=True,verbose=1)  
start_time=time.time()
history = model.fit(train_X,train_y, steps_per_epoch=steps_per_epoch, epochs=EPOCHS,validation_data=(test_X,test_y),batch_size=BATCH_SIZE,callbacks=[mc])
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from tensorflow.keras.models import load_model
model=load_model('plant_gpu.h5')
c=0
t=0
for i in range(len(test_X1)):
  test_image = np.expand_dims(test_X1[i], axis = 0)
  result=model.predict(test_image)
  if result[0]<0.5 and test_y1[i]==0:
    c+=1
  if result[0]>=0.5 and test_y1[i]==1:
    c+=1
  t+=1

In [ ]:
print(c/t)